In [2]:
import dlc_bci as bci
import torch
from torch.autograd import Variable
from random import randint
from torch import nn
from torch import optim
from torch import Tensor
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load the data

In [3]:
train_input , train_target = bci.load ( root = "./ data_bci")
test_input , test_target = bci.load ( root = "./ data_bci " , train = False )

train_target = Variable(train_target)
train_input = Variable(train_input.unsqueeze(dim=2))
test_input = Variable(test_input.unsqueeze(dim=2))
test_target = Variable(test_target)

mu, std = train_input.data.mean(), train_input.data.std() 
train_input.data.sub_(mu).div_(std)
test_input.data.sub_(mu).div_(std)


print('train_input size:' , train_input.size())
print ( 'train_target size:' , train_target.size())
print ('test_input size:' , test_input.size())
print ('test_target size:' , test_target.size())

('train_input size:', torch.Size([316, 28, 1, 50]))
('train_target size:', torch.Size([316]))
('test_input size:', torch.Size([100, 28, 1, 50]))
('test_target size:', torch.Size([100]))


## Preparing the data

In [6]:
class Dataset:
    def __init__(self,train_data,train_target):
        self.train_data = train_data.data
        self.train_target = train_target
    def __getitem__(self, index):
        return self.train_data[index], self.train_target[index]
    def __len__(self):
        return len(self.train_target)

## Convolutional neural network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d (28, 32, kernel_size = (1,5))    # after conv: 20*32*1*46
        self.avg_pool1 = nn.AvgPool2d(kernel_size = (1,4))      # size -> 1/4 *size
        self.conv2 = nn.Conv2d (32 , 64, kernel_size = (1,5))   
        self.fc1 = nn.Linear (448 , 50)                         # size : 64 ->32 => 448/2
        self.dr = nn.Dropout(p=0.8)
        self.fc2 = nn.Linear (50 , 2)
        
    def forward (self , x):
        x = self.conv1(x)
        #print ('after first conv',x.size())
        x = F.relu(self.avg_pool1(x))
        x = F.relu(self.conv2(x))
        x = x.view (-1, 448)
        x = self.dr(x)
        x = F.relu ( self.fc1 (x))
        x = self.dr(x)
        x = self.fc2(x)
                   #x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride =3) )
        #x = F.relu (F.max_pool2d ( self.conv2(x), kernel_size =2, stride =2) )
        return x


In [ ]:
# The function computes the accuracy
def accuracy(output,target):
    return (output.float()==target.float()).float().sum()/len(output)

In [ ]:
data = Dataset(train_input,train_target)
dataset_loader = torch.utils.data.DataLoader(data,
                                             batch_size = 40, shuffle=True,
                                             num_workers=4)

In [9]:
### train the model
net = Net()
epoch = 1000
batch_size = 20
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.01,momentum = 0.5)



i = 0
for e in range(epoch):
    for b in range(0,train_input.size(0),batch_size):
        input_ = train_input.narrow(0,b,min(batch_size,train_input.size(0)-b))
        target = train_target.narrow(0,b,min(batch_size,train_input.size(0)-b))
        output = net(input_).view(min(batch_size,train_input.size(0)-b),2)
        loss = criterion(output,target)
        net.zero_grad()
        loss.backward()
        optimizer.step()
    if e%10 == 0: # print the loss every 10 epochs
        output = net(test_input).view(-1,2)
        a,predicted_class = output.max(dim=1)
        output_train = net(train_input).view(-1,2)
        a,predicted_class_train = output.max(dim=1)
        
        #predicted_class = 1*(predicted_class.float().mean(dim=1) > 0.5)
        print(e//10,'loss train:',criterion(output_train,train_target).data[0],'loss test:',criterion(output,test_target).data[0],' accuracy test:',accuracy(predicted_class,test_target).data[0])

(0, 'loss train:', 0.6936902403831482, 'loss test:', 0.6947332620620728, ' accuracy test:', 0.44999998807907104)
(1, 'loss train:', 0.7020915746688843, 'loss test:', 0.6980192065238953, ' accuracy test:', 0.47999998927116394)
(2, 'loss train:', 0.69966721534729, 'loss test:', 0.6979005336761475, ' accuracy test:', 0.5099999904632568)
(3, 'loss train:', 0.6976155042648315, 'loss test:', 0.7213453054428101, ' accuracy test:', 0.5)
(4, 'loss train:', 0.6964417099952698, 'loss test:', 0.6897997260093689, ' accuracy test:', 0.5600000023841858)
(5, 'loss train:', 0.6948499083518982, 'loss test:', 0.7155714631080627, ' accuracy test:', 0.4000000059604645)
(6, 'loss train:', 0.6868491172790527, 'loss test:', 0.6975237131118774, ' accuracy test:', 0.4699999988079071)
(7, 'loss train:', 0.6775312423706055, 'loss test:', 0.707350492477417, ' accuracy test:', 0.44999998807907104)
(8, 'loss train:', 0.6778134107589722, 'loss test:', 0.6980429291725159, ' accuracy test:', 0.5299999713897705)
(9, 'lo

(73, 'loss train:', 0.1832265704870224, 'loss test:', 1.0660862922668457, ' accuracy test:', 0.7900000214576721)
(74, 'loss train:', 0.15368202328681946, 'loss test:', 1.238193392753601, ' accuracy test:', 0.7099999785423279)
(75, 'loss train:', 0.16754645109176636, 'loss test:', 1.2689553499221802, ' accuracy test:', 0.7300000190734863)
(76, 'loss train:', 0.37511715292930603, 'loss test:', 2.415447473526001, ' accuracy test:', 0.699999988079071)
(77, 'loss train:', 0.12390346825122833, 'loss test:', 1.5053437948226929, ' accuracy test:', 0.699999988079071)
(78, 'loss train:', 0.17766223847866058, 'loss test:', 0.8611512184143066, ' accuracy test:', 0.6800000071525574)
(79, 'loss train:', 0.07056857645511627, 'loss test:', 1.5393235683441162, ' accuracy test:', 0.7400000095367432)
(80, 'loss train:', 0.11096471548080444, 'loss test:', 1.3536381721496582, ' accuracy test:', 0.699999988079071)
(81, 'loss train:', 0.19003765285015106, 'loss test:', 1.4711185693740845, ' accuracy test:', 

## Data Augmentaiotn: Averaging two datapoints in the same class

In [10]:
class0 = train_input[(train_target == 0).nonzero(),]
class1 = train_input[(train_target == 1).nonzero(),]
class0.squeeze_(dim=1)
class1.squeeze_(dim=1)
N = class0.shape[0]
train_input_aug0 = [(class0[i]+class0[j])/2 for i in range(N) for j in range(i,N)]
train_input_aug0 = torch.stack(train_input_aug0)
train_out_aug0 = Variable(Tensor(train_input_aug0.shape[0]).fill_(0))
N = class1.shape[0]
train_input_aug1 = [(class1[i]+class1[j])/2 for i in range(N) for j in range(i,N)]
train_input_aug1 = torch.stack(train_input_aug1)
train_out_aug1 = Variable(Tensor(train_input_aug1.shape[0]).fill_(1))

###
train_aug = torch.cat((train_input_aug0,train_input_aug1))
train_target_aug = torch.cat((train_out_aug0,train_out_aug1)).data.long()

mu, std = train_aug.data.mean(), train_aug.data.std() ;
train_aug.data.sub_(mu).div_(std);
test_input.data.sub_(mu).div_(std);

In [11]:
type(train_aug)

torch.autograd.variable.Variable

In [12]:
data = Dataset(train_aug,train_target_aug)
dataset_loader = torch.utils.data.DataLoader(data,
                                             batch_size = 40, shuffle=True,
                                             num_workers=4)

In [13]:

net = Net()

In [14]:

### train the model
epoch = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.01,momentum = 0.5)
i = 0
for e in range(epoch):
    for input_, target_ in dataset_loader:
        input_ = Variable(input_)
        target_ = Variable(target_)
        output = net(input_).view(-1,2)
        loss = criterion(output,target_) 
        net.zero_grad()
        loss.backward()
        optimizer.step()
    if e%1 == 0: # print the loss every 10 epochs
        output = net(test_input).view(-1,2)
        a,predicted_class = output.max(dim=1)
        output_train = net(train_input).view(-1,2)
        a,predicted_class_train = output.max(dim=1)
        #predicted_class = 1*(predicted_class.float().mean(dim=1) > 0.5)
        print(e//1,'loss train:',criterion(output_train,train_target).data[0],'loss test:',criterion(output,test_target).data[0],' accuracy test:',accuracy(predicted_class,test_target).data[0])

(0, 'loss train:', 0.6843971610069275, 'loss test:', 0.7019619941711426, ' accuracy test:', 0.5099999904632568)
(1, 'loss train:', 0.6991146802902222, 'loss test:', 0.7469719052314758, ' accuracy test:', 0.5699999928474426)
(2, 'loss train:', 0.9319414496421814, 'loss test:', 1.0630385875701904, ' accuracy test:', 0.5799999833106995)
(3, 'loss train:', 0.5540997385978699, 'loss test:', 0.7864527106285095, ' accuracy test:', 0.6700000166893005)
(4, 'loss train:', 0.43757498264312744, 'loss test:', 0.7659677267074585, ' accuracy test:', 0.75)
(5, 'loss train:', 0.5060749650001526, 'loss test:', 0.9224059581756592, ' accuracy test:', 0.7099999785423279)
(6, 'loss train:', 0.410137802362442, 'loss test:', 0.741931676864624, ' accuracy test:', 0.7900000214576721)
(7, 'loss train:', 0.31868189573287964, 'loss test:', 0.9916681051254272, ' accuracy test:', 0.75)
(8, 'loss train:', 0.32711607217788696, 'loss test:', 0.8552981615066528, ' accuracy test:', 0.7799999713897705)
(9, 'loss train:', 

## Adding L1 reguralizer

In [15]:
def l1_reguralizer(paramters):
    loss = 0
    for p in paramters:
        loss += torch.abs(p).mean()
    return loss

In [ ]:
net = Net()

In [ ]:

### train the model
epoch = 100
lambda_ = 10**-2
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.01,momentum = 0.5)
i = 0
loss_train = []
loss_test = []
accuracy_train = []
accuracy_test = []
for e in range(epoch):
    if e == 30:
        optimizer = optim.SGD(net.parameters(),lr = 0.003,momentum = 0.5)
    if e== 60:
        optimizer = optim.SGD(net.parameters(),lr = 0.001,momentum = 0.5)
    for input_, target_ in dataset_loader:
        input_ = Variable(input_)
        target_ = Variable(target_)
        output = net(input_).view(-1,2)
        loss = criterion(output,target_) + lambda_ * l1_reguralizer(net.parameters())
        net.zero_grad()
        loss.backward()
        optimizer.step()
        
        
    if e%1 == 0: # print the loss every 10 epochs
        output = net(test_input).view(-1,2)
        a,predicted_class = output.max(dim=1)
        output_train = net(train_input).view(-1,2)
        a,predicted_class_train = output_train.max(dim=1)
        accuracy_train.append(accuracy(predicted_class_train,train_target).data[0])
        accuracy_test.append(accuracy(predicted_class,test_target).data[0])
        loss_train.append(criterion(output_train,train_target).data[0]+ lambda_ * l1_reguralizer(net.parameters()).data[0])
        loss_test.append(criterion(output,test_target).data[0]+ lambda_ * l1_reguralizer(net.parameters()).data[0])
        print(e//1,'loss train:',round(loss_train[-1],2), 'accuracy train:', round(accuracy_train[-1],2),\
              'loss test:',round(loss_test[-1],2),' accuracy test:',round(accuracy_test[-1],2))

(0, 'loss train:', 0.69, 'accuracy train:', 0.56, 'loss test:', 0.72, ' accuracy test:', 0.49)
(1, 'loss train:', 0.68, 'accuracy train:', 0.63, 'loss test:', 0.87, ' accuracy test:', 0.55)
(2, 'loss train:', 0.55, 'accuracy train:', 0.73, 'loss test:', 0.75, ' accuracy test:', 0.73)
(3, 'loss train:', 0.52, 'accuracy train:', 0.76, 'loss test:', 0.9, ' accuracy test:', 0.7)
(4, 'loss train:', 0.43, 'accuracy train:', 0.81, 'loss test:', 0.76, ' accuracy test:', 0.71)
(5, 'loss train:', 0.41, 'accuracy train:', 0.84, 'loss test:', 1.03, ' accuracy test:', 0.72)
(6, 'loss train:', 0.38, 'accuracy train:', 0.82, 'loss test:', 0.97, ' accuracy test:', 0.71)
(7, 'loss train:', 0.37, 'accuracy train:', 0.84, 'loss test:', 1.13, ' accuracy test:', 0.7)
(8, 'loss train:', 1.4, 'accuracy train:', 0.7, 'loss test:', 2.79, ' accuracy test:', 0.56)
(9, 'loss train:', 0.69, 'accuracy train:', 0.81, 'loss test:', 1.39, ' accuracy test:', 0.78)
(10, 'loss train:', 0.28, 'accuracy train:', 0.89, 'los

In [ ]:
plt.plot(loss_train)
plt.plot(loss_test)

In [ ]:
plt.plot(accuracy_train)
plt.plot(accuracy_test)

## Average between 3

In [ ]:
class0 = train_input[(train_target == 0).nonzero(),]
class1 = train_input[(train_target == 1).nonzero(),]
class0.squeeze_(dim=1)
class1.squeeze_(dim=1)

N = class0.shape[0]
train_input_aug0 = [(class0[randint(0,N-1)]+class0[randint(0,N-1)]+ class0[randint(0,N-1)])/3 for i in range(200*10**3)]
train_input_aug0 = torch.stack(train_input_aug0)
train_out_aug0 = Variable(Tensor(train_input_aug0.shape[0]).fill_(0))
N = class1.shape[0]
train_input_aug1 = [(class0[randint(0,N-1)]+class0[randint(0,N-1)]+ class0[randint(0,N-1)])/3 for i in range(200*10**3)]
train_input_aug1 = torch.stack(train_input_aug1)
train_out_aug1 = Variable(Tensor(train_input_aug1.shape[0]).fill_(1))

###
train_aug = torch.cat((train_input_aug0,train_input_aug1))
train_target_aug = torch.cat((train_out_aug0,train_out_aug1)).data.long()

In [ ]:
data = Dataset(train_aug,train_target_aug)
dataset_loader = torch.utils.data.DataLoader(data,
                                             batch_size = 100, shuffle=True,
                                             num_workers=4)

In [ ]:
net = Net()

In [ ]:
### train the model
epoch = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.0001,momentum = 0.5)
i = 0
for e in range(epoch):
    for input_, target_ in dataset_loader:
        input_ = Variable(input_)
        target_ = Variable(target_)
        output = net(input_).view(-1,2)
        loss = criterion(output,target_)
        net.zero_grad()
        loss.backward()
        optimizer.step()
    if e%1 == 0: # print the loss every 10 epochs
        output = net(test_input).view(-1,2)
        a,predicted_class = output.max(dim=1)
        output_train = net(train_input).view(-1,2)
        a,predicted_class_train = output.max(dim=1)
        #predicted_class = 1*(predicted_class.float().mean(dim=1) > 0.5)
        print(e//1,'loss train:',criterion(output_train,train_target).data[0],'loss test:',criterion(output,test_target).data[0],' accuracy test:',accuracy(predicted_class,test_target).data[0])